In [ ]:
# Importar librerías para tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd  # Pandas es una poderosa librería para manipulación y análisis de datos en Python.

# Importar librerías para web scraping y manipulación de datos
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup
import requests

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.

# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

# Configuraciones
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

import numpy as np

import re

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
from selenium.webdriver.common.by import By

In [ ]:
df_comedy = pd.read_csv("comedy.csv", index_col = 0)

In [ ]:
comedy_df = df_comedy.values.tolist()

In [ ]:
comedy_df

In [ ]:
lista_peliculas_comedy = []

for pelicula in comedy_df:
    lista_peliculas_comedy.append (pelicula[1])

In [ ]:
len(lista_peliculas_comedy)

In [ ]:
lista_peliculas_comedy_1 = lista_peliculas_comedy[:500]

In [ ]:
len(lista_peliculas_comedy_1)

In [ ]:
#funcion para optener el reparto de la peli desde google

def obtener_actores(lista):
    
    diccionario = {}
    
    driver = webdriver.Chrome() 

    url = "https://www.google.com/" 

    driver.get(url)  

    driver.maximize_window()

    sleep(1)

    driver.find_element("css selector", "#L2AGLb > div").click() # Aceptar cookies

    sleep(1)

    lista_actores = []

    for indice, pelicula in enumerate(lista):
        lista_actores = []

        driver.find_element("css selector", "#APjFqb").send_keys(f"reparto `{pelicula}`", Keys.ENTER) 

        sleep(4)

        try:
            for num in range(1, 9):
                nombre_actor = driver.find_element("css selector", f"#rcnt > div.M8OgIe > div > div > div > div > div > div.uciohe > div:nth-child({num})").text.split("\n")[0]
                lista_actores.append(nombre_actor)

        except:
            pass

        sleep(4)

        driver.back()

        # creo un diccionario con la peli y sus actores pero solo lo apendeo si he detectado 1 o mas actores
        if len(lista_actores) > 0:
            dic_peli = {pelicula: lista_actores}

            #añado esa peli al diccionario final
            diccionario.update(dic_peli)
        
        #muestro los resultados
        print(indice)
        print(pelicula)
        print(lista_actores)
        print("----------")

        if indice % 50 == 0 : #cada 50 películas cierro el navegador y vuelvo a empezar para que no pete
            
            print("reinicio navegador") 

            driver.quit()

            driver = webdriver.Chrome() 

            url = "https://www.google.com/" 

            driver.get(url)  

            driver.maximize_window()

            sleep(1)

            driver.find_element("css selector", "#L2AGLb > div").click() # Aceptar cookies

            sleep(1)
        

    #driver.quit()  # Cerrar el navegador al finalizar
    return diccionario

In [ ]:
# Llamamos a la función para sacar los actores de cada pelicula

resultados = obtener_actores(lista_peliculas_comedy_1)
print(resultados)

In [ ]:
len(resultados)

In [ ]:
#los he puesto de la manera que queriamos ver la tabla
lista_formato_tabla = []

In [ ]:
for pelicula in resultados:
    diccionario_nuevo = {"pelicula" : pelicula, "actor" : resultados[pelicula][0]}
    listaa_prueba.append(diccionario_nuevo)
    diccionario_nuevo = {"pelicula" : pelicula, "actor" : resultados[pelicula][1]}
    listaa_prueba.append(diccionario_nuevo)
    diccionario_nuevo = {"pelicula" : pelicula, "actor" : resultados[pelicula][2]}
    listaa_prueba.append(diccionario_nuevo)
    diccionario_nuevo = {"pelicula" : pelicula, "actor" : resultados[pelicula][3]}
    listaa_prueba.append(diccionario_nuevo)
    diccionario_nuevo = {"pelicula" : pelicula, "actor" : resultados[pelicula][4]}
    listaa_prueba.append(diccionario_nuevo)
    diccionario_nuevo = {"pelicula" : pelicula, "actor" : resultados[pelicula][5]}
    listaa_prueba.append(diccionario_nuevo)
    diccionario_nuevo = {"pelicula" : pelicula, "actor" : resultados[pelicula][6]}
    listaa_prueba.append(diccionario_nuevo)
    diccionario_nuevo = {"pelicula" : pelicula, "actor" : resultados[pelicula][7]}
    lista_formato_tabla.append(diccionario_nuevo)

In [ ]:
len(lista_formato_tabla)

In [ ]:
resultados_nombre = pd.DataFrame(lista_formato_tabla)

In [ ]:
resultados_nombre

In [ ]:
resultados_nombre.to_csv("pelis_actores_500.csv")

In [ ]:
#con lo obtenido en la anterior búsqueda creamos una lista de actores sin duplicados 
actores = []
resultados2 = resultados.values()
resultados2

In [ ]:
for indice, pelicula in enumerate(resultados2):
   for actor in pelicula:
       if actor not in actores:
           actores.append(actor)

In [ ]:
len(actores) 

In [ ]:
#modificamos la lista poniendo barrabajas a los espacios para crear las url de wikipedia
actores_mod = []

In [ ]:
for actor in actores:
    actor2 = actor.replace(" ", "_")
    actores_mod.append(actor2)

In [ ]:
#buscamos cada actor en wikipedia para sacar sus datos biográficos
actores_datos = []
for indice, actor in enumerate(actores_mod):
    print(indice)
    try:

        url_actor = f"https://es.wikipedia.org/wiki/{actor}"

        respuesta= requests.get(url_actor)

        # vemos si todo ha ido bien
        print("La respuesta de la petición es:", respuesta.status_code)
        
        if respuesta.status_code == 200: 
            premios_pre = []
            # nombre
            nombre = actor.replace("_", " ")
            print (f"El nombre es {nombre}")

            # creamos el objeto BeautifulSoup para poder acceder al contenido solicitado
            sopa = BeautifulSoup(respuesta.content, 'html.parser')

            #nacimiento = sopa.find_all("td")
            #print(nacimiento[3])


            #buscamos tablas
            tabla_bio = sopa.find("table")

            # separar la tabla en una lista con distintos datos separados
            celdas = tabla_bio.find_all('td')

            # Extraer el texto de cada celda y dividirlo por espacios
            lista_resultados_pre = [celda.get_text(strip=True, separator=' ') for celda in celdas]
            print(lista_resultados_pre)

            
            meses = ["enero", "febrero", "marzo", "abril", "mayo", "junio", "julio", "agosto", "septiembre", "octubre", "noviembre", "diciembre"]
            for elemento in lista_resultados_pre:
                #activo desde:
                patron = re.compile(r"^desde\s\d{4}$")
                if patron.search(elemento):
                    activo = elemento.split()[-1]
                    print(f"Activo desde: {activo}")

                # año nacimiento
                for mes in meses:
                    if mes in elemento:
                        fecha_nacimiento = elemento.split("(")[0]
                        print(f"La fecha de nacimiento es: {fecha_nacimiento}")

                        #pais de nacimiento
                        pais_nacimiento = elemento.split()[-1]
                        if pais_nacimiento == "Unidos":
                            pais_nacimiento = "Estados Unidos"
                        if pais_nacimiento == "Unido":
                            pais_nacimiento = "Reino Unido"
                        if pais_nacimiento == "Dominicana":
                            pais_nacimiento = "Republica Dominicana"
                        if pais_nacimiento == "Rico":
                            pais_nacimiento = "Puerto Rico"
                        print(f"El país de nacimiento es: {pais_nacimiento}")

                #premios
                premios_lista = ["Oscar", "Globo de Oro", "BAFTA", "León de Oro","Palma de Oro","Oso de Oro","SAG","Critics' Choice","Goya","Oso de Plata","Emmy", "Tony","Concha de Plata","Premio ", "Mejor", "Award"]
                
                for premio in premios_lista:
                    if premio in elemento: 
                        premios_pre.append(elemento)
                        premios = premios_pre

            for elemento in premios:
                if "Sindicato" in elemento:
                    premios.remove(elemento)

            print(f"Los premios son: {premios}")
            actor_datos = {"nombre": nombre, "fecha_nacimiento": fecha_nacimiento, "pais_nacimiento" : pais_nacimiento, "activo desde" : activo, "premios": premios}
            actores_datos.append(actor_datos)
            print("-----------------")

        else:
            print("sin wikipedia")
    except:
        print("--------------")
    


    print(actores_datos)